In [1]:
# MOD 1: needed to append sys path to import holodeck
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck')


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# Frequency Bins
Choose the edges of the frequency bins at which to calculate the GWB

In [7]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
print('dur=%e cad=%e' % (dur,cad))
fobs = utils.nyquist_freqs(dur,cad)
print('fobs=', fobs)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print('fobs_edges=',fobs_edges)
# print(f"Number of frequency bins: {fobs.size-1}")
# print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
# print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

dur=5.000000e+07 cad=5.000000e+06
fobs= [2.e-08 4.e-08 6.e-08 8.e-08 1.e-07]
fobs_edges= [1.0e-08 3.0e-08 5.0e-08 7.0e-08 9.0e-08 1.2e-07]


### Nyquist Freq Calculation

In [8]:
# utils.nyquist_freqs code
fmin = 1.0 / dur
fmax = 1.0 / cad * 0.5
print('fmin', fmin, 'fmax', fmax)
# df = fmin / sample
df = fmin    # bin width
freqs = np.arange(fmin, fmax + df/10.0, df)   # centers
print(freqs)
freqs_edges = freqs - df/2.0    # shift to edges
freqs_edges = np.concatenate([freqs_edges, [fmax + df/2]])
print(freqs_edges)

fmin 2e-08 fmax 1e-07
[2.e-08 4.e-08 6.e-08 8.e-08 1.e-07]
[1.0e-08 3.0e-08 5.0e-08 7.0e-08 9.0e-08 1.1e-07]


In [9]:
#calculator cell
print(1/dur)
print(1/2/cad)
print(YR)
width = 1/dur
print(width)
print('freqs\n', 1/dur, 1/dur+width, 
      1/dur+2*width,
      1/dur+3*width, 
      1/dur+4*width)
print(fmax+df/10)
print('freqs edges\n(%e,%e,%e,%e,%e,%e)' %(1/dur-width/2,
      1/dur+width-width/2,
      1/dur+2*width-width/2,
      1/dur+3*width-width/2,
      1/dur+4*width-width/2,
      1/2/cad+width/2))


2e-08
1e-07
31557600.0
2e-08
freqs
 2e-08 4e-08 6.000000000000001e-08 8e-08 1e-07
1.02e-07
freqs edges
(1.000000e-08,3.000000e-08,5.000000e-08,7.000000e-08,9.000000e-08,1.100000e-07)


# SAM

Construct a semi-analytic-model

## SAM Edges
mtot, mrat, and redz

In [10]:
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
print(MSOL)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
# gsmf=GSMF_Schechter, 
# gpf=GPF_Power_Law, 
# gmt=GMT_Power_Law, 
# mmbulge=relations.MMBulge_MM2013
   
print('sam.mtot (M_sun)',sam.mtot/MSOL,
'\nsam.mrat',sam.mrat,
'\nsam.redz',sam.redz)
print('sam.mtot',sam.mtot)
print(sam.edges)
print(MSOL)

1.988409870698051e+33
sam.mtot (M_sun) [  502914.421587    5029144.21587004 50291442.15870041] 
sam.mrat [0.1 1. ] 
sam.redz [0.001 0.01  0.1   1.   ]
sam.mtot [1.e+39 1.e+40 1.e+41]
[array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ])]
1.988409870698051e+33


In [11]:
param_names = ['mtot', 'mrat', 'redz']
params = [mtot, mrat, redz]
shape=None

        # Process grid specifications

for ii, (par, name) in enumerate(zip(params, param_names)):
            log.debug(f"{name}: {par}")
            if isinstance(par, tuple) and (len(par) == 3):
                print('a')
                continue
            elif isinstance(par, np.ndarray):
                print('b')
                continue
            else:
                err = (
                    f"{name} (type={type(par)}, len={len(par)}) must be a (3,) tuple specifying a log-spacing, "
                    "or ndarray of grid edges!"
                )
                log.exception(err)
                print(err)
                raise ValueError(err)
# skip the "if shape is not none" stuff

# Set grid-spacing for each parameter
for ii, (par, name) in enumerate(zip(params, param_names)):
    print('t1', par, name)
    log.debug(f"{name}: {par}")
    if isinstance(par, tuple) and (len(par) == 3):
        print('t2')
        par = np.logspace(*np.log10(par[:2]), par[2])
    elif isinstance(par, np.ndarray):
        print('t3')
        par = np.copy(par)
    else:
        print('t4')
        err = f"{name} must be a (3,) tuple specifying a log-spacing; or ndarray of grid edges!  ({par})"
        log.exception(err)
        raise ValueError(err)

    log.debug(f"{name}: [{par[0]}, {par[-1]}] {par.size}")
    params[ii] = par

    
# determine grid spacing
for ii, (par, name) in enumerate(zip(params, param_names)):
        log.debug(f"{name}: {par}")
        print('par, name, ii=\n     ',par, name, ii)
if isinstance(par, tuple) and (len(par) == 3):
    par = np.logspace(*np.log10(par[:2]), par[2])
    print('tuple name, par:',name, par)
elif isinstance(par, np.ndarray):
    par = np.copy(par)
    print('ndarray name, par:',name, par)
else:
    err = f"{name} must be a (3,) tuple specifying a log-spacing; or ndarray of grid edges!  ({par})"
    log.exception(err)
    print('ndarray name, par:',name, par)
    raise ValueError(err)

    log.debug(f"{name}: [{par[0]}, {par[-1]}] {par.size}")
    params[ii] = par
    # print(params)


a
a
a
t1 (1e+39, 1e+41, 3) mtot
t2
t1 (0.1, 1.0, 2) mrat
t2
t1 (0.001, 1.0, 4) redz
t2
par, name, ii=
      [1.e+39 1.e+40 1.e+41] mtot 0
par, name, ii=
      [0.1 1. ] mrat 1
par, name, ii=
      [0.001 0.01  0.1   1.   ] redz 2
ndarray name, par: redz [0.001 0.01  0.1   1.   ]


# gwb

In [12]:
fobs_gw_edges = freqs_edges
print('fobs_gw_edges', fobs_gw_edges)

gwb = sam.gwb(fobs_gw_edges, realize=False)    # calculate many different realizations


fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges)
print('fobs_gw_cents', fobs_gw_cents)
# ---- Get the differential-number of binaries for each bin
# convert to orbital-frequency (from GW-frequency)
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0
print('fobs_orb_edges', fobs_orb_edges)
print('fobs_orb_cents', fobs_orb_cents)

fobs_gw_edges [1.0e-08 3.0e-08 5.0e-08 7.0e-08 9.0e-08 1.1e-07]
fobs_gw_cents [2.e-08 4.e-08 6.e-08 8.e-08 1.e-07]
fobs_orb_edges [5.0e-09 1.5e-08 2.5e-08 3.5e-08 4.5e-08 5.5e-08]
fobs_orb_cents [1.e-08 2.e-08 3.e-08 4.e-08 5.e-08]


## gwb: dynamic_binary_number

### gwb: dynamic_binary_number: static_binary_density

#### stellar masses

In [50]:
# total-mass, mass-ratio ==> (M1, M2)
masses = utils.m1m2_from_mtmr(sam.mtot[:, np.newaxis], sam.mrat[np.newaxis, :])
print(masses.shape)
print('m1',masses[0,:])
print('m2',masses[1,:])
# print(masses[:,2])
print('ratios', np.array(masses[1,:])/np.array(masses[0,:]))
print('totals', np.array(masses[1,:])+np.array(masses[0,:]))
# BH-masses to stellar-masses
masses = sam._mmbulge.mstar_from_mbh(masses, scatter=False)
print('primary masses: ',masses[0,:])
print('secondary masses: ',masses[1,:])
print('ratios', np.array(masses[1,:])/np.array(masses[0,:]))
print('totals', np.array(masses[0,:])+np.array(masses[1,:]))


(2, 3, 2)
m1 [[9.09090909e+38 5.00000000e+38]
 [9.09090909e+39 5.00000000e+39]
 [9.09090909e+40 5.00000000e+40]]
m2 [[9.09090909e+37 5.00000000e+38]
 [9.09090909e+38 5.00000000e+39]
 [9.09090909e+39 5.00000000e+40]]
ratios [[0.1 1. ]
 [0.1 1. ]
 [0.1 1. ]]
totals [[1.e+39 1.e+39]
 [1.e+40 1.e+40]
 [1.e+41 1.e+41]]
primary masses:  [[6.96723892e+41 3.94263955e+41]
 [6.24369466e+42 3.53319842e+42]
 [5.59529010e+43 3.16627753e+43]]
secondary masses:  [[7.77463038e+40 3.94263955e+41]
 [6.96723892e+41 3.53319842e+42]
 [6.24369466e+42 3.16627753e+43]]
ratios [[0.1115884 1.       ]
 [0.1115884 1.       ]
 [0.1115884 1.       ]]
totals [[7.74470196e+41 7.88527911e+41]
 [6.94041855e+42 7.06639683e+42]
 [6.21965957e+43 6.33255507e+43]]


#### static_binary_density

In [51]:
# `mstar_tot` starts as the secondary mass, sorry
mstar_pri, mstar_tot = masses
# q = m2 / m1
mstar_rat = mstar_tot / mstar_pri
print('mstar_rat', mstar_rat)
# M = m1 + m2
mstar_tot = mstar_pri + mstar_tot
print('mstar_tot', mstar_tot)
redz = sam.redz[np.newaxis, np.newaxis, :]
print('redz', redz)
args = [mstar_pri[..., np.newaxis], mstar_rat[..., np.newaxis], mstar_tot[..., np.newaxis], redz]
# Convert to shape (M, Q, Z)
mstar_pri, mstar_rat, mstar_tot, redz = np.broadcast_arrays(*args)
# print('mstar_pri', mstar_pri.shape, mstar_pri)
# print('mstar_rat', mstar_rat.shape, mstar_rat)
print('mstar_tot\n', mstar_tot.shape, '\n', mstar_tot)
# print('redz', redz)

mstar_rat [[0.1115884 1.       ]
 [0.1115884 1.       ]
 [0.1115884 1.       ]]
mstar_tot [[7.74470196e+41 7.88527911e+41]
 [6.94041855e+42 7.06639683e+42]
 [6.21965957e+43 6.33255507e+43]]
redz [[[0.001 0.01  0.1   1.   ]]]
mstar_tot
 (3, 2, 4) 
 [[[7.74470196e+41 7.74470196e+41 7.74470196e+41 7.74470196e+41]
  [7.88527911e+41 7.88527911e+41 7.88527911e+41 7.88527911e+41]]

 [[6.94041855e+42 6.94041855e+42 6.94041855e+42 6.94041855e+42]
  [7.06639683e+42 7.06639683e+42 7.06639683e+42 7.06639683e+42]]

 [[6.21965957e+43 6.21965957e+43 6.21965957e+43 6.21965957e+43]
  [6.33255507e+43 6.33255507e+43 6.33255507e+43 6.33255507e+43]]]


In [52]:
# each of these could otherwise use MTOT
mass_gsmf = mstar_pri
mass_gpf = mstar_pri
mass_gmt = mstar_pri

# gsmf=GSMF_Schechter, 
# gpf=GPF_Power_Law, 
# gmt=GMT_Power_Law, 
# mmbulge=relations.MMBulge_MM2013

zprime = sam._gmt.zprime(mass_gmt, mstar_rat, redz)
print(zprime)

[[[-1.         -1.          0.05842088  0.90887098]
  [-1.         -1.          0.05842088  0.90887098]]

 [[-1.         -1.          0.05842088  0.90887098]
  [-1.         -1.          0.05842088  0.90887098]]

 [[-1.         -1.          0.05842088  0.90887098]
  [-1.         -1.          0.05842088  0.90887098]]]


##### GSMF_Schecter

calls _gsmf(mass_gsmf, redz) with mass_gsmf=mstar_pri

$$ \mathrm{GSMF} = \Phi(M,z) = \frac{dn_G}{d\log_{10}(M)} = \ln(10)*\phi * \frac{m_\mathrm{star}}{m_\mathrm{mref}}^{1+\alpha}\times e^\frac{m_\mathrm{star}}{m_\mathrm{mref}}$$

In [55]:
# testing GSMF dependence on phi
print(mstar_pri)
# print(mstar_pri[:,0,:])
test_mstar = np.copy(mstar_pri[:,:,:])
for q in range(len(test_mstar[0,:,0])):
    test_mstar[:,q,:] = (mstar_pri[:,0,:])

print(np.shares_memory(mstar_pri, test_mstar))
print('\n',test_mstar)
# print(mstar_pri)

# impact of q occurs in applying the mmbulge relation to 
# mpri and msec to get mstar_pri and mstar_sec
print('gsmf',
      sam._gsmf(test_mstar, redz))

[[[6.96723892e+41 6.96723892e+41 6.96723892e+41 6.96723892e+41]
  [3.94263955e+41 3.94263955e+41 3.94263955e+41 3.94263955e+41]]

 [[6.24369466e+42 6.24369466e+42 6.24369466e+42 6.24369466e+42]
  [3.53319842e+42 3.53319842e+42 3.53319842e+42 3.53319842e+42]]

 [[5.59529010e+43 5.59529010e+43 5.59529010e+43 5.59529010e+43]
  [3.16627753e+43 3.16627753e+43 3.16627753e+43 3.16627753e+43]]]
False

 [[[6.96723892e+41 6.96723892e+41 6.96723892e+41 6.96723892e+41]
  [6.96723892e+41 6.96723892e+41 6.96723892e+41 6.96723892e+41]]

 [[6.24369466e+42 6.24369466e+42 6.24369466e+42 6.24369466e+42]
  [6.24369466e+42 6.24369466e+42 6.24369466e+42 6.24369466e+42]]

 [[5.59529010e+43 5.59529010e+43 5.59529010e+43 5.59529010e+43]
  [5.59529010e+43 5.59529010e+43 5.59529010e+43 5.59529010e+43]]]
gsmf [[[0.01730046 0.01723279 0.01657041 0.01119722]
  [0.01730046 0.01723279 0.01657041 0.01119722]]

 [[0.01005741 0.01001214 0.00957046 0.00609531]
  [0.01005741 0.01001214 0.00957046 0.00609531]]

 [[0.005145

In [31]:
# return the number density of galaxies at a given stellar mass
my_gsmf = sam._gsmf(mass_gsmf, redz)
print('\nGSMF_Schechter(mass_gsmf, redz):\n', my_gsmf)
# `gsmf` returns [1/Mpc^3]   `dtdz` returns [sec]
# dens = self._gsmf(mass_gsmf, redz) * self._gpf(mass_gpf, mstar_rat, redz) * cosmo.dtdz(redz)
# `gmt` returns [sec]
# dens /= self._gmt(mass_gmt, mstar_rat, redz)
# now `dens` is  ``dn_gal / [dlog10(Mstar) dq_gal dz]``  with units of [Mpc^-3]


GSMF_Schechter(mass_gsmf, redz):
 [[[0.01730046 0.01723279 0.01657041 0.01119722]
  [0.01985139 0.01977678 0.01904587 0.01306936]]

 [[0.01005741 0.01001214 0.00957046 0.00609531]
  [0.01162106 0.01157053 0.01107713 0.00716418]]

 [[0.00514529 0.0051191  0.00486439 0.00291996]
  [0.00632837 0.00629712 0.005993   0.00365316]]]


In [12]:
print('mass_gmt=mstar=:\n', mass_gmt)
print('redz:\n', redz)

mstar = mass_gmt
phi0 = -2.77            # - 2.77  +/- [-0.29, +0.27]  [1/Mpc^3]
phiz = -0.27            # - 0.27  +/- [-0.21, +0.23]  [1/Mpc^3]
mref0 = 10**11.24 * MSOL     # +11.24  +/- [-0.17, +0.20]  Msol
mrefz = 0               #  0.0                        Msol    # noqa
alpha0 = -1.24          # -1.24   +/- [-0.16, +0.16]
alphaz = -0.03          # -0.03   +/- [-0.14, +0.16]

phi = np.power(10, phi0 + phiz*redz)
print('phi:\n', phi)

mref = mref0 + mrefz*redz
print('mref:\n', mref)
print('mref from sam.py\n', sam._gsmf._mref_func(redz))

alpha = alpha0 + alphaz*redz
print('alpha:\n', alpha)

xx = mstar / mref
print('mstar/mref\n', xx)
# [Chen2019]_ Eq.8
rv = np.log(10.0) * phi * np.power(xx, 1.0 + alpha) * np.exp(-xx)
print('rv', rv)

mass_gmt=mstar=:
 [[[6.96723892e+41 6.96723892e+41 6.96723892e+41 6.96723892e+41]
  [3.94263955e+41 3.94263955e+41 3.94263955e+41 3.94263955e+41]]

 [[6.24369466e+42 6.24369466e+42 6.24369466e+42 6.24369466e+42]
  [3.53319842e+42 3.53319842e+42 3.53319842e+42 3.53319842e+42]]

 [[5.59529010e+43 5.59529010e+43 5.59529010e+43 5.59529010e+43]
  [3.16627753e+43 3.16627753e+43 3.16627753e+43 3.16627753e+43]]]
redz:
 [[[0.001 0.01  0.1   1.   ]
  [0.001 0.01  0.1   1.   ]]

 [[0.001 0.01  0.1   1.   ]
  [0.001 0.01  0.1   1.   ]]

 [[0.001 0.01  0.1   1.   ]
  [0.001 0.01  0.1   1.   ]]]
phi:
 [[[0.00169719 0.00168772 0.00159588 0.00091201]
  [0.00169719 0.00168772 0.00159588 0.00091201]]

 [[0.00169719 0.00168772 0.00159588 0.00091201]
  [0.00169719 0.00168772 0.00159588 0.00091201]]

 [[0.00169719 0.00168772 0.00159588 0.00091201]
  [0.00169719 0.00168772 0.00159588 0.00091201]]]
mref:
 [[[3.45546032e+44 3.45546032e+44 3.45546032e+44 3.45546032e+44]
  [3.45546032e+44 3.45546032e+44 3.45546

In [13]:
# calculator
# first numbers
print(np.log(10)*.00169*(6.18*10**40)**(1+-1.24003)*np.exp(6.19*10**40))

print('should be')
print('\nGSMF_Schechter(mass_gsmf, redz):\n', my_gsmf)

print('first element', (np.log(10) * 0.00169*(0.0020163)**(1-1.24003)
                        *np.e**(-0.0020163)))

inf
should be

GSMF_Schechter(mass_gsmf, redz):
 [[[0.01730046 0.01723279 0.01657041 0.01119722]
  [0.01985139 0.01977678 0.01904587 0.01306936]]

 [[0.01005741 0.01001214 0.00957046 0.00609531]
  [0.01162106 0.01157053 0.01107713 0.00716418]]

 [[0.00514529 0.0051191  0.00486439 0.00291996]
  [0.00632837 0.00629712 0.005993   0.00365316]]]
first element 0.017227186392488002


##### GPF_Power_Law

In [14]:
# Return the fraction of galaxies in pairs of the given parameters.
my_gpf = sam._gpf(mass_gpf, mstar_rat, redz)
print('GPF_Power_Law(mass_gpt, mstar_rat, redz)', my_gpf)
mref=None
mref_log10=11.0
mref, _ = utils._parse_val_log10_val_pars(
            mref, mref_log10, val_units=MSOL, name='mref', only_one=True
        )
print(mref)
print(10**11*MSOL)

f0p = 0.025/(1-.25)
print(f0p-1/30)


GPF_Power_Law(mass_gpt, mstar_rat, redz) [[[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]

 [[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]

 [[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]]
1.9884098706980508e+44
1.9884098706980508e+44
0.0


In [15]:
print('rv by hand:\n', 1/30 * (1+redz)**0.8)

rv by hand:
 [[[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]

 [[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]

 [[0.03336    0.03359973 0.03597434 0.0580367 ]
  [0.03336    0.03359973 0.03597434 0.0580367 ]]]


##### dzdt cosmology

In [16]:
import cosmopy
from astropy import cosmology as cmy
cosmo = cosmopy.Cosmology(h=.6933, Om0=.2880, Ob0=.0472)
print(cosmo)

my_dtdz = cosmo.dtdz(redz)
print('cosmo.dtdz(redz):\n', my_dtdz)
z=redz[0,0,0]
print('z =',z)
efuncz = cosmo.efunc(z)
print('cosmo.efunc(z) =', efuncz)
# print('Ogamma0', cmy.FlatLambdaCDM.Ogamma0.__dir__())
# print('Ogamma0', cmy.FlatLambdaCDM.Ogamma0.__eq__)
OmegaLam0 = .6911
Eofz = np.sqrt(.2880 * (1+z)**3 + OmegaLam0)
print('E(z) by hand =', Eofz)
hubbletime = 13.799*10**9 * 365 * 24 * 3600 #s
lookbacktime = hubbletime/(1+z)/efuncz
print('differnetial lookback time:', lookbacktime)


<class 'cosmopy.cosmology.Cosmology'> :: H0 = 69.33000000 km / (Mpc s), Om0 = 0.28800000, Ob0 = 0.04720000
cosmo.dtdz(redz):
 [[[4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]
  [4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]]

 [[4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]
  [4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]]

 [[4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]
  [4.44434276e+17 4.38754128e+17 3.86602417e+17 1.28139693e+17]]]
z = 0.001
cosmo.efunc(z) = 1.0004323386856304
E(z) by hand = 0.9899317472876602
differnetial lookback time: 4.345426638623889e+17


##### GMT_Power_Law

In [17]:
gmtmref = 1*10**11 * .4 / .6933 * MSOL
print(gmtmref)
mtime = .55 * GYR * 1 * (1+redz)**(-.5)
print('mtime by hand:\n', mtime)
print(mass_gmt.shape)
print( mstar_rat.shape)
print(redz.shape)
my_gmt = sam._gmt(mass_gmt, mstar_rat, redz)
print('mtime by sam:\n', my_gmt)

1.1472146953399976e+44
mtime by hand:
 [[[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]

 [[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]

 [[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]]
(3, 2, 4)
(3, 2, 4)
(3, 2, 4)
mtime by sam:
 [[[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]

 [[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]

 [[1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]
  [1.73480082e+16 1.72705421e+16 1.65489451e+16 1.22730261e+16]]]


#### static_binary_density continued

In [18]:
my_dens_gal = my_gsmf * my_gpf * my_dtdz / my_gmt
print('my_dens_gal:\n', my_dens_gal) # `dn_gal / [dlog10(Mstar) dq_gal dz]``  with units of [Mpc^-3]

# variables:
print('mstar_rat %f, \nmstar_tot %e, \nmstar_pri %e,\
        \nmstar_sec %e, \nredz %f, \nmtot %e, \
        \nmrat %f'
        % (mstar_rat[0,0,0], mstar_tot[0,0,0], 
        mstar_pri[0,0,0], 
        mstar_tot[0,0,0]-mstar_pri[0,0,0], 
        redz[0,0,0], mtot[0], mrat[0]))

my_dens_gal:
 [[[0.01478569 0.01470979 0.01392581 0.00678492]
  [0.01696582 0.01688133 0.01600619 0.00791934]]

 [[0.00859548 0.00854629 0.00804304 0.00369344]
  [0.00993184 0.00987653 0.00930924 0.00434111]]

 [[0.00439738 0.00436963 0.00408804 0.00176934]
  [0.00540848 0.00537518 0.00503653 0.00221362]]]
mstar_rat 0.111588, 
mstar_tot 7.744702e+41, 
mstar_pri 6.967239e+41,        
mstar_sec 7.774630e+40, 
redz 0.001000, 
mtot 1.000000e+39,         
mrat 0.100000


In [19]:
#dq_bh/dq_gal
mplaw = sam._mmbulge._mplaw
print(mplaw)
my_dqbh_dqgal = mplaw * np.power(mstar_rat, mplaw - 1.0)
print(my_dqbh_dqgal)
print(1.05*mstar_rat[0,0,0]**.05)          

1.05
[[[0.94095803 0.94095803 0.94095803 0.94095803]
  [1.05       1.05       1.05       1.05      ]]

 [[0.94095803 0.94095803 0.94095803 0.94095803]
  [1.05       1.05       1.05       1.05      ]]

 [[0.94095803 0.94095803 0.94095803 0.94095803]
  [1.05       1.05       1.05       1.05      ]]]
0.9409580270439349


In [20]:
# dmstar_tot/dmbh_tot
dmstar_dmbh_pri = sam._mmbulge.dmstar_dmbh(mstar_pri)   # [unitless]
print(dmstar_dmbh_pri)
qterm = (1.0 + mstar_rat) / (1.0 + sam.mrat[np.newaxis, :, np.newaxis])
dmstar_dmbh = dmstar_dmbh_pri * qterm
qterm = (1.0 + mstar_rat) / (1.0 + sam.mrat[np.newaxis, :, np.newaxis])
print('qterm:\n', qterm)
dmstar_dmbh = dmstar_dmbh_pri * qterm
print('dmstar/dmbh:\n', dmstar_dmbh)
print('dmstar/dmbh by hand:',729.9*1.0105)

[[[729.90122025 729.90122025 729.90122025 729.90122025]
  [750.97896261 750.97896261 750.97896261 750.97896261]]

 [[654.1013449  654.1013449  654.1013449  654.1013449 ]
  [672.99017429 672.99017429 672.99017429 672.99017429]]

 [[586.17324855 586.17324855 586.17324855 586.17324855]
  [603.1004825  603.1004825  603.1004825  603.1004825 ]]]
qterm:
 [[[1.01053491 1.01053491 1.01053491 1.01053491]
  [1.         1.         1.         1.        ]]

 [[1.01053491 1.01053491 1.01053491 1.01053491]
  [1.         1.         1.         1.        ]]

 [[1.01053491 1.01053491 1.01053491 1.01053491]
  [1.         1.         1.         1.        ]]]
dmstar/dmbh:
 [[[737.59066275 737.59066275 737.59066275 737.59066275]
  [750.97896261 750.97896261 750.97896261 750.97896261]]

 [[660.99224265 660.99224265 660.99224265 660.99224265]
  [672.99017429 672.99017429 672.99017429 672.99017429]]

 [[592.34853004 592.34853004 592.34853004 592.34853004]
  [603.1004825  603.1004825  603.1004825  603.1004825 ]]]


In [21]:
dqbh_dqgal = my_dqbh_dqgal
my_dens = (my_dens_gal * sam.mtot[:, np.newaxis, np.newaxis] 
            * dmstar_dmbh / dqbh_dqgal / mstar_tot)
print(my_dens)
print(sam._density) # this accounts for the zprimes<0,
                    # otherwise they match
zprime = sam._gmt.zprime(mass_gmt, mstar_rat, redz)
# find valid entries (M, Q, Z)
bads = (zprime < 0.0)
my_dens[bads] = 0.0  
print('my_dens - bads:\n', my_dens)  

[[[0.01496519 0.01488837 0.01409486 0.00686729]
  [0.0153885  0.01531186 0.01451809 0.00718308]]

 [[0.00869982 0.00865004 0.00814068 0.00373827]
  [0.00900847 0.0089583  0.00844376 0.00393752]]

 [[0.00445076 0.00442267 0.00413767 0.00179082]
  [0.00490565 0.00487545 0.00456828 0.00200782]]]
[[[0.         0.         0.01409486 0.00686729]
  [0.         0.         0.01451809 0.00718308]]

 [[0.         0.         0.00814068 0.00373827]
  [0.         0.         0.00844376 0.00393752]]

 [[0.         0.         0.00413767 0.00179082]
  [0.         0.         0.00456828 0.00200782]]]
my_dens - bads:
 [[[0.         0.         0.01409486 0.00686729]
  [0.         0.         0.01451809 0.00718308]]

 [[0.         0.         0.00814068 0.00373827]
  [0.         0.         0.00844376 0.00393752]]

 [[0.         0.         0.00413767 0.00179082]
  [0.         0.         0.00456828 0.00200782]]]


## dynamic_binary_number cont.

In [23]:
edges = sam.edges + [fobs_orb_cents]

dc = cosmo.comoving_distance(redz).to('Mpc').value
# print('dc:\n', dc)
from holodeck.constants import SPLC, MPC
cosmo_fact = 4 * np.pi * (SPLC/MPC) * np.square(dc) * (1.0 + redz)
# print('cosmo_fact:\n', cosmo_fact)

mchirp = utils.m1m2_from_mtmr(sam.mtot[:, np.newaxis], sam.mrat[np.newaxis, :])
mchirp = utils.chirp_mass(*mchirp)
# (M, Q, 1, 1) make shape broadcastable for later calculations
mchirp = mchirp[..., np.newaxis, np.newaxis]
# print('mchirp:\n',mchirp)
print(np.shape(mtot), np.shape(mrat))

(3,) (3,)


In [24]:
mt, mr, rz = [gg.ravel() for gg in sam.grid]
print(mt.shape, mt)
print(mr.shape, mr)
print(rz.shape, rz)

(24,) [1.e+39 1.e+39 1.e+39 1.e+39 1.e+39 1.e+39 1.e+39 1.e+39 1.e+40 1.e+40
 1.e+40 1.e+40 1.e+40 1.e+40 1.e+40 1.e+40 1.e+41 1.e+41 1.e+41 1.e+41
 1.e+41 1.e+41 1.e+41 1.e+41]
(24,) [0.1 0.1 0.1 0.1 1.  1.  1.  1.  0.1 0.1 0.1 0.1 1.  1.  1.  1.  0.1 0.1
 0.1 0.1 1.  1.  1.  1. ]
(24,) [0.001 0.01  0.1   1.    0.001 0.01  0.1   1.    0.001 0.01  0.1   1.
 0.001 0.01  0.1   1.    0.001 0.01  0.1   1.    0.001 0.01  0.1   1.   ]


In [25]:
fobs_orb = fobs_orb_cents
print(fobs_orb)
frst_orb = fobs_orb[:, np.newaxis] * (1.0 + rz[np.newaxis, :])
print('\nfrst_orb\n', frst_orb.shape)
sa = utils.kepler_sepa_from_freq(mt[np.newaxis, :], frst_orb)
print('\nsa\n', sa.shape)
dadt = holo.hardening.Hard_GW.dadt(mt[np.newaxis, :], mr[np.newaxis, :], sa)
print('\ndadt\n', dadt.shape)     

[1.e-08 2.e-08 3.e-08 4.e-08 5.e-08]

frst_orb
 (5, 24)

sa
 (5, 24)

dadt
 (5, 24)


In [26]:
dfdt, frst_orb = utils.dfdt_from_dadt(dadt, sa, 
                                    frst_orb=frst_orb)
print('dfdt\n', dfdt.shape)
print('frst_orb\n', frst_orb.shape)
tau = frst_orb / dfdt
print('tau\n',tau.shape)
# (X, M*Q*Z) ==> (M*Q*Z, X)
tau = np.moveaxis(tau, 0, -1)
print(tau.shape)
# (M*Q*Z, X) ==> (M, Q, Z, X)
xsize = fobs_orb.size
tau = tau.reshape(my_dens.shape + (xsize,))
print(tau.shape)

dfdt
 (5, 24)
frst_orb
 (5, 24)
tau
 (5, 24)
(24, 5)
(3, 2, 4, 5)


In [27]:
dnum = my_dens * cosmo_fact
print(cosmo_fact.shape)
print(my_dens.shape)
dnum = dnum[..., np.newaxis] * tau
edges = [sam.mtot, sam.mrat, sam.redz] + [fobs_orb, ]
print('%e' % np.max(dnum))
print('dnum\n', dnum)
print('edges\n', edges)

(3, 2, 4)
(3, 2, 4)
6.639770e+09
dnum
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [5.84719522e+08 9.20875543e+07 3.12337585e+07 1.45028810e+07
    7.99885055e+06]
   [6.63976979e+09 1.04569822e+09 3.54674264e+08 1.64687149e+08
    9.08307731e+07]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.99099790e+08 3.13562520e+07 1.06352439e+07 4.93830025e+06
    2.72364681e+06]
   [2.29589815e+09 3.61581301e+08 1.22639190e+08 5.69454865e+07
    3.14074449e+07]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [7.27579957e+06 1.14586663e+06 3.88648844e+05 1.80462686e+05
    9.95315380e+04]
   [7.78703039e+07 1.22638044e+

# gwb cont.
## number grid integrated

Using values calculated thus far

In [35]:
edges[-1] = fobs_orb_edges

# correct by using sam.py version of edges
# hard = holo.hardening.Hard_GW
# edges, dnum = sam.dynamic_binary_number(hard, 
#                 fobs_orb=fobs_orb_cents)
print(dnum.shape)
print(edges[-1].shape)

edges[-1]=fobs_orb_edges
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
print(number.shape)
number = number * np.diff(np.log(fobs_gw_edges))
print(number.shape)
print(number)
# print('edges\n', edges)
# print(len(edges[-1]))
# print(edges[-1])
# print(edges[0])
# print(edges[1])
for num in range(len(fobs_gw_cents)):
    print('\nfobs_gw_cents: ', fobs_gw_cents[num])
    print('fobs_orb_cents: ', fobs_orb_cents)
    print('frequency bin %f nHz to %f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    print('number =\n', number[:,:,:,num])
print(len(edges[-1]))


(3, 2, 4, 5)
(6,)
(2, 1, 3, 5)
(2, 1, 3, 5)
[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.20574395e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    2.73337920e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.09892126e+05 8.04726202e+03 1.79782747e+03 6.23514038e+02
    2.74590854e+02]
   [1.28993591e+07 9.44603826e+05 2.11032610e+05 7.31893338e+04
    3.22320276e+04]]]]

fobs_gw_cents:  2e-08
fobs_orb_cents:  [1.e-08 2.e-08 3.e-08 4.e-08 5.e-08]
frequency bin 5.000000 nHz to 15.000000 nHz
number =
 [[[0.00000000e+00 8.82745690e+06 1.09390698e+09]]

 [[0.00000000e+00 1.09892126e+05 1.28993591e+07]]]

fobs_gw_cents:  4.000000000000001e-08
fobs_orb_cents:  [1.e-08 2.e-08 3.e-08 4.e-08 5.e-08]
frequency bin 15.000000 nHz to 25.000000 nHz
number =
 [[[0.00000000e+00 6.46423553e+05 8.01054307e+07]]

 [[0.00000000

Using: my_edges

In [39]:
# matching edges4, dnum4, edges 5
hard = holo.hardening.Hard_GW
my_edges, my_dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
# equivalent to
my_edges[-1] = fobs_orb_edges
my_edges_direct = [edges[0], edges[1], edges[2], fobs_orb_edges]

print(np.array([my_edges]) - np.array([my_edges_direct])) # = 0

my_number = utils._integrate_grid_differential_number(my_edges, my_dnum, freq=False)
print(my_number.shape)
my_number = my_number * np.diff(np.log(fobs_gw_edges))
print(my_number.shape)
my_hc = holo.gravwaves._gws_from_number_grid_integrated(my_edges, my_number, realize=False)
print(my_hc)

####           worked!

[[array([0., 0., 0.]) array([0., 0.]) array([0., 0., 0., 0.])
  array([0., 0., 0., 0., 0., 0.])]]
(2, 1, 3, 5)
(2, 1, 3, 5)
[8.94836712e-16 5.43608499e-16 4.12359473e-16 3.39693273e-16
 2.92461393e-16]


## get strain

using values calculated thus far, no realize

In [45]:
# print(number)
hc = holo.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False)
print(hc)
for ii in range(len(hc)):
    print('frequency bin %.1f nHz to %.1f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    print('\t h_c =', hc[ii])

[8.94836712e-16 5.43608499e-16 4.12359473e-16 3.39693273e-16
 2.92461393e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 8.948367120399258e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 5.436084991203219e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 4.123594726001719e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 3.396932730091485e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 2.9246139304080997e-16


using my_edges, no realize

In [46]:
my_hc = holo.gravwaves._gws_from_number_grid_integrated(my_edges, my_number, realize=False)
print(my_hc)
for ii in range(len(my_hc)):
    print('frequency bin %.1f nHz to %.1f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    print('\t h_c =', my_hc[ii]) 

[8.94836712e-16 5.43608499e-16 4.12359473e-16 3.39693273e-16
 2.92461393e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 8.948367120399258e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 5.436084991203219e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 4.123594726001719e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 3.396932730091485e-16
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = 2.9246139304080997e-16


with realizations:

In [47]:
my_realized_hc = holo.gravwaves._gws_from_number_grid_integrated(my_edges, my_number, realize=3)
for ii in range(len(my_realized_hc)):
    print('frequency bin %.1f nHz to %.1f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    print('\t h_c =', my_realized_hc[ii])

frequency bin 45.0 nHz to 55.0 nHz
	 h_c = [8.95411963e-16 8.95030718e-16 8.95269487e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = [5.42389980e-16 5.44134908e-16 5.43346805e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = [4.14354910e-16 4.14754593e-16 4.12855271e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = [3.41030236e-16 3.39016573e-16 3.38579851e-16]
frequency bin 45.0 nHz to 55.0 nHz
	 h_c = [2.92589856e-16 2.94227966e-16 2.88130754e-16]


#### troubleshooting edges

In [ ]:
edges2 = [sam.mtot, sam.mrat, sam.redz] + [fobs_orb]
edges4, dnum4 = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
print(np.array([edges4]) - np.array([edges2]))
print(np.array([edges4]) - np.array([edges]))

In [ ]:
# hc = holo.gravwaves._gws_from_number_grid_integrated(edges2, number, realize=False)
# print('check1')
edges4[-1] = fobs_orb_edges
edges5 = [edges[0], edges[1], edges[2], fobs_orb_edges]
print(np.array([edges4]) - np.array([edges5]))
# print(np.array([edges4]) - np.array([edges5]))

# number4 = utils._integrate_grid_differential_number(edges4, dnum4, freq=False)
# print(number4.shape)
# number4 = number4 * np.diff(np.log(fobs_gw_edges))
# print(number4.shape)
# hc = holo.gravwaves._gws_from_number_grid_integrated(edges4, number4, realize=False)
# print(hc)

number5 = utils._integrate_grid_differential_number(edges5, dnum4, freq=False)
print(number5.shape)
number5 = number5 * np.diff(np.log(fobs_gw_edges))
print(number5.shape)
hc5 = holo.gravwaves._gws_from_number_grid_integrated(edges5, number5, realize=False)
print(hc5)

####           worked!

[[array([0., 0., 0.]) array([0., 0.]) array([0., 0., 0., 0.])
  array([0., 0., 0., 0., 0., 0.])]]
(2, 1, 3, 5)
(2, 1, 3, 5)
err here
hc:
 [[[[8.52704562e-38 4.29736171e-37 1.10683151e-36 2.16573459e-36
    3.64526073e-36]
   [9.29179258e-40 4.68277002e-39 1.20609755e-38 2.35996821e-38
    3.97218545e-38]
   [1.98216207e-41 9.98947086e-41 2.57289515e-40 5.03437784e-40
    8.47362365e-40]]]


 [[[1.83709629e-34 9.25838514e-34 2.38459620e-33 4.66593373e-33
    7.85347616e-33]
   [2.00185603e-36 1.00887222e-35 2.59845839e-35 5.08439738e-35
    8.55781413e-35]
   [4.27043872e-38 2.15216625e-37 5.54313457e-37 1.08462383e-36
    1.82558687e-36]]]] 
hc shape: (2, 1, 3, 5)
number:
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.20574395e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    2.73337920e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+0

In [ ]:
# this worked in hc for dummies:

sams_edges, sams_dnum = sam.dynamic_binary_number(hard, 
                        fobs_orb_cents)
sams_edges[-1] = fobs_orb_edges
sams_number = utils._integrate_grid_differential_number(sams_edges, sams_dnum, freq=False)
sams_number = sams_number * np.diff(np.log(fobs_gw_edges))
log.debug(f"number: {utils.stats(sams_number)}")
log.debug(f"number.sum(): {sams_number.sum():.4e}")
sams_hc = holo.sam.gravwaves._gws_from_number_grid_integrated(sams_edges, sams_number, realize=False)
print(sams_hc.shape, sams_hc)

err here
hc:
 [[[[8.52704562e-38 4.29736171e-37 1.10683151e-36 2.16573459e-36
    3.64526073e-36]
   [9.29179258e-40 4.68277002e-39 1.20609755e-38 2.35996821e-38
    3.97218545e-38]
   [1.98216207e-41 9.98947086e-41 2.57289515e-40 5.03437784e-40
    8.47362365e-40]]]


 [[[1.83709629e-34 9.25838514e-34 2.38459620e-33 4.66593373e-33
    7.85347616e-33]
   [2.00185603e-36 1.00887222e-35 2.59845839e-35 5.08439738e-35
    8.55781413e-35]
   [4.27043872e-38 2.15216625e-37 5.54313457e-37 1.08462383e-36
    1.82558687e-36]]]] 
hc shape: (2, 1, 3, 5)
number:
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.20574395e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    2.73337920e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.09892126e+05 8.04726202e+03 1.79782747e+03 6.23514038e+02
    2.74590854e+02]
   [1.28993591e+07

In [ ]:
# compare successful version from hc for dummies
# to my values here

# good: sams_edges, sams_dnum, sams_number, sams_hc



Plot GWB over multiple realizations

In [ ]:
# nsamp = 5    # number of sample GWB spectra to plot
# fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

# # `fobs` are bin centers in CGS units, convert to [1/yr]
# xx = fobs * YR

# # plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
# yy = 1e-15 * np.power(xx, -2.0/3.0)
# ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)

# # Plot the median GWB spectrum
# ax.plot(xx, np.median(gwb, axis=-1), 'k-')

# # Plot `nsamp` random spectra 
# nsamp = np.min([nsamp, gwb.shape[1]])
# idx = np.random.choice(gwb.shape[1], nsamp, replace=False)
# ax.plot(xx, gwb[:, idx], 'k-', lw=1.0, alpha=0.1)

# # plot contours at 50% and 98% confidence intervals
# for pp in [50, 98]:
#     percs = pp / 2
#     percs = [50 - percs, 50 + percs]
#     ax.fill_between(xx, *np.percentile(gwb, percs, axis=-1), alpha=0.25, color='b')
    
# plt.show()

# build SAM component-by-component

Construct the four components required for all SAM models:

1) **Galaxy Stellar Mass Function (GSMF)**: number-density of galaxies as a function of stellar mass
2) **Galaxy Pair Fraction (GPF)**: fraction of galaxies that are in pairs
3) **Galaxy Merger Time (GMT)**: time it takes for galaxies to merge
4) **MBH––Galaxy Scaling Relationship (e.g. Mbh-Mbulge)**: mapping between galaxy properties (i.e. stellar mass) and BH mass

In [ ]:
# gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
# gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
# gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
# mmbulge = holo.relations.MMBulge_MM2013() # M-MBulge Relation            (MMB)

Build SAM using these components

In [ ]:
# sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)

Calculate the distribution of GWB Amplitudes at 1/yr

In [ ]:
# fobs_yr = 1.0/YR
# fobs_yr = fobs_yr * (1.0 + np.array([-0.05, 0.05]))
# ayr = sam.gwb(fobs_yr, realize=100)
# gwb_ref = sam.gwb_ideal(1.0/YR)

Plot the distribution

In [ ]:
# fig, ax = plt.subplots(figsize=[8, 4])
# ax.set(xlabel=r'$\log_{10}(A_\mathrm{yr})$', ylabel='Probability Density')
# ax.grid(alpha=0.2)

# # use `kalepy` do draw the 1D distribution
# kale.dist1d(np.log10(ayr), density=True, confidence=True)
# ax.axvline(np.log10(gwb_ref), ls='--', color='k')

# plt.show()

## Plot GWB Amplitude Distribution vs. M-MBulge parameters

Calculate GWB amplitudes at $f = 1/yr$ over a grid of M-Mbulge parameters, specifically the amplitude and power-law.

In [ ]:
# # Choose parameters to explore
# NREALS = 30     # number of realizations at each parameter combination
# alpha_list = [0.75, 1.0, 1.25, 1.5]     # M-Mbulge power-law index
# norm_list = np.logspace(8, 9.5, 4)     # M-Mbulge normalization, units of [Msol]

# dist_mmb = np.zeros((len(alpha_list), norm_list.size, NREALS))

# # Iterate over a grid of both paremeters
# for aa, alpha in enumerate(tqdm.tqdm(alpha_list)):
#     for nn, norm in enumerate(tqdm.tqdm(norm_list, leave=False)):
#         # Create the M-Mbulge relationship for these parameters
#         mmbulge = holo.relations.MMBulge_Standard(mamp=norm*MSOL, mplaw=alpha)
#         # Build a new sam
#         sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=30)
#         # Calculate the distribution of GWB amplitudes
#         dist_mmb[aa, nn, :] = sam.gwb(fobs_yr, realize=NREALS)

Plot the interquartile ranges for each power-law, as a function of normalization

In [ ]:
# fig, ax = plt.subplots(figsize=[6, 4])
# ax.set(xscale='log', xlabel='M-MBulge Mass Normalization', yscale='log', ylabel=r'GWB Amplitude $A_\mathrm{yr}$')
# ax.grid(alpha=0.2)

# for aa, dd in zip(alpha_list, dist_mmb):
#     med = np.median(dd, axis=-1)
#     cc, = ax.plot(norm_list, med, label=aa)
#     cc = cc.get_color()
#     ax.fill_between(norm_list, *np.percentile(dd, [25, 75], axis=-1), color=cc, alpha=0.15)
    
# plt.legend(title='M-MBulge Slope')
# plt.show()

# Generate Full Population of Binaries

In [ ]:
# # Choose a hardening model (determines number of binaries in each frequency bin)
# hard = holo.evolution.Hard_GW()

In [ ]:
# # Sample the SAM population using 'outlier sampling'
# # use orbital frequency
# fobs_orb_edges = fobs_edges / 2.0
# vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(sam, hard, fobs_orb=fobs_orb_edges, sample_threshold=1e2, poisson_inside=True, poisson_outside=True)

In [ ]:
# # use GW frequency
# gff, gwf, gwb = holo.gravwaves._gws_from_samples(vals, weights, fobs_edges)

In [ ]:
# fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
# ax.scatter(gff, gwf)
# plot.draw_hist_steps(ax, fobs_edges, gwb, yfilter=lambda xx: np.greater(xx, 0.0))

# plt.show()

In [ ]:
# hs, fo = holo.gravwaves._strains_from_samples(vals)

In [ ]:
# nloud = 5
# colors = plot._get_cmap('plasma')(np.linspace(0.05, 0.95, nloud))# print(colors)

# fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
# for ii in range(fobs_edges.size-1):
#     # if ii < 10 or ii > 16:
#     #     continue
    
#     fextr = [fobs_edges[ii+jj] for jj in range(2)]
#     fextr = np.asarray(fextr)
#     cycles = 1.0 / np.diff(np.log(fextr))[0]

#     idx = (fextr[0] <= fo) & (fo < fextr[1])
#     hs_bin = hs[idx]
#     fo_bin = fo[idx]    
#     ww_bin = weights[idx]
#     ww = ww_bin * cycles

#     tot = np.sqrt(np.sum(ww * hs_bin**2))
#     ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

#     idx = np.argsort(hs_bin)[::-1]
#     if any(ww_bin[idx[:nloud]] > 1):
#         raise
    
#     for jj, cc in enumerate(colors):
#         if jj >= len(idx):
#             break
#         hi = idx[jj]
#         lo = idx[jj+1:]
#         gw_hi = np.sqrt(np.sum(ww[hi] * hs_bin[hi]**2))
#         gw_lo = np.sqrt(np.sum(ww[lo] * hs_bin[lo]**2))

#         fave = np.average(fo_bin[hi], weights=hs_bin[hi])
#         ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
#         ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

# plt.show()